# Demand Forecasting Model Training

This notebook trains time series models to predict material demand:
- ARIMA (AutoRegressive Integrated Moving Average)
- LSTM (Long Short-Term Memory)
- Prophet (Facebook's forecasting tool)
- Simple Moving Average (baseline)

## Approach
We'll compare multiple forecasting methods and select the best performing one.


In [ ]:
# Install required dependencies (run this first)
!pip install pandas numpy scikit-learn matplotlib seaborn statsmodels prophet torch


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Set paths
BASE_DIR = Path('../')
DATA_DIR = BASE_DIR / 'data'
MODELS_DIR = BASE_DIR / 'models' / 'demand_forecaster'
MODELS_DIR.mkdir(parents=True, exist_ok=True)

print(f"Base directory: {BASE_DIR}")
print(f"Data directory: {DATA_DIR}")
print(f"Models directory: {MODELS_DIR}")


## 1. Generate Sample Time Series Data


In [ ]:
# Generate sample demand data (daily orders for 90 days)
dates = pd.date_range(start='2024-01-01', periods=90, freq='D')
np.random.seed(42)

# Create realistic demand pattern with trend and seasonality
base_demand = 100
trend = np.linspace(0, 20, 90)  # Increasing trend
seasonality = 10 * np.sin(2 * np.pi * np.arange(90) / 7)  # Weekly pattern
noise = np.random.normal(0, 5, 90)  # Random noise
demand = base_demand + trend + seasonality + noise
demand = np.maximum(demand, 0)  # No negative demand

df = pd.DataFrame({
    'date': dates,
    'demand': demand.astype(int),
    'material_id': 'MAT-001'
})

print(f"✓ Generated {len(df)} days of demand data")
print(f"\nFirst few rows:")
print(df.head())
print(f"\nStatistics:")
print(df['demand'].describe())

# Plot
plt.figure(figsize=(12, 6))
plt.plot(df['date'], df['demand'])
plt.title('Historical Demand Data')
plt.xlabel('Date')
plt.ylabel('Demand (units)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(MODELS_DIR / 'demand_history.png')
plt.show()


## 2. Simple Moving Average Forecast


In [ ]:
# Simple Moving Average (baseline)
window = 7  # 7-day moving average
df['ma_forecast'] = df['demand'].rolling(window=window).mean()

# Forecast next 7 days
last_avg = df['demand'].tail(window).mean()
forecast_dates = pd.date_range(start=df['date'].max() + timedelta(days=1), periods=7, freq='D')
forecast = pd.DataFrame({
    'date': forecast_dates,
    'forecast': [last_avg] * 7
})

print(f"Last {window}-day average: {last_avg:.2f}")
print(f"\n7-day forecast:")
print(forecast)

# Plot
plt.figure(figsize=(12, 6))
plt.plot(df['date'], df['demand'], label='Historical', alpha=0.7)
plt.plot(df['date'], df['ma_forecast'], label='Moving Average', linestyle='--')
plt.plot(forecast['date'], forecast['forecast'], label='Forecast', marker='o')
plt.title('Demand Forecast - Moving Average')
plt.xlabel('Date')
plt.ylabel('Demand (units)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(MODELS_DIR / 'ma_forecast.png')
plt.show()

# Save forecast
forecast.to_csv(MODELS_DIR / 'ma_forecast.csv', index=False)
print(f"\n✓ Forecast saved to {MODELS_DIR}")
